In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
intervalls = np.array([1, 2, 3, 4, 5, 6, 8, 10, 15, 20, 25, 30, 35, 40, 50, 60, 80, 100,  125, 150, 200, 300, 500, 750, 1000])

In [ ]:
runs_per_dataset = 10

In [ ]:
# Pre train
import random
import tables 
import numpy as np
import matplotlib.pyplot as plt

lcs = []
for i in range(0, 40):
    lcs.append([])
windows_data = 0
max_i = 0
max_v = 0
i = 0
labels = []
data = []
 
with tables.open_file('pathtofile.h5', mode='r') as hdf5_file:
    print("shape: ", hdf5_file.root.train.shape)
    for window in hdf5_file.root.train:
        #print(window.shape)
        #print(window[0, 4:15])
        label = window[-1, 9].astype(int)
        #print(window.shape)
        lcs[label].append(window)
        #print(label)
        labels.append(label)
hist = plt.hist(labels, bins= range(0,41))
plt.hist(labels, bins= range(0,41))
print("Number of occ: ", hist[0])
ps = np.min(hist[0])/ hist[0]

In [ ]:
eqltrain = []
for data_label_i in lcs:
    for j in range(0, 4000):
        eqltrain.append(data_label_i[j])
print(len(eqltrain))      

In [ ]:
# Pre train
import random
import tables 
import numpy as np
import matplotlib.pyplot as plt

lcs = []
for i in range(0, 40):
    lcs.append([])
windows_data = 0
max_i = 0
max_v = 0
i = 0
labels = []
data = []
 
with tables.open_file('pathtofile.h5', mode='r') as hdf5_file:
    print("shape: ", hdf5_file.root.train.shape)
    for window in hdf5_file.root.train:
        #print(window.shape)
        #print(window[0, 4:15])
        label = window[-1, 9].astype(int)
        #print(window.shape)
        lcs[label].append(window)
        #print(label)
        labels.append(label)
hist = plt.hist(labels, bins= range(0,41))
plt.hist(labels, bins= range(0,41))
print("Number of occ: ", hist[0])
ps = np.min(hist[0])/ hist[0]

In [ ]:
eqltest = []
for data_label_i in lcs:
    for j in range(0, 500):
        eqltest.append(data_label_i[j])
print(len(eqltest))      

In [ ]:
vision_bins = 16
window_size = 40
img_dtype = tables.Float64Atom()  # dtype in which the images will be saved
data_shape = (0, window_size -1, 5 + 6 + 4 + (vision_bins * 8))
with tables.open_file('pathtofile.h5', mode='w') as hdf5_file:
    hdf5_file.create_earray(hdf5_file.root, 'train', img_dtype, shape = data_shape)
    print(hdf5_file.root.train.shape)
    for d in eqltest:
        hdf5_file.root.train.append(d[None, :, :])
    hdf5_file.close()    

In [ ]:
def iterate_data(data, batchsize, shuffle=False):
    if shuffle:
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, data.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield data[excerpt, :, :]

In [ ]:
#from scipy, quelle noch angeben:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(15,15))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.autograd import Variable
from torch.distributions import Categorical
import numpy as np
import math

def log_sum_exp(x, dim=1):
    x_max, x_argmax = x.max(dim, keepdim=True)
    x_max_broadcast = x_max.expand(*x.size())
    return x_max + torch.log(
        torch.sum(torch.exp(x - x_max_broadcast), dim=dim, keepdim=True))

class MDN(nn.Module):
    """A mixture density network layer
    The input maps to the parameters of a MoG probability distribution, where
    each Gaussian has O dimensions and diagonal covariance.
    Arguments:
        in_features (int): the number of dimensions in the input
        out_features (int): the number of dimensions in the output
        num_gaussians (int): the number of Gaussians per output dimensions
    Input:
        minibatch (BxD): B is the batch size and D is the number of input
            dimensions.
    Output:
        (pi, sigma, mu) (BxG, BxGxO, BxGxO): B is the batch size, G is the
            number of Gaussians, and O is the number of dimensions for each
            Gaussian. Pi is a multinomial distribution of the Gaussians. Sigma
            is the standard deviation of each Gaussian. Mu is the mean of each
            Gaussian.
    """

    def __init__(self, in_features, out_features, num_gaussians):
        super(MDN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_gaussians = num_gaussians
        self.pi = nn.Sequential(
            nn.Linear(in_features, num_gaussians), nn.LogSoftmax(dim=1))
        self.sigma = nn.Linear(in_features, out_features * num_gaussians)
        self.mu = nn.Linear(in_features, out_features * num_gaussians)

    def forward(self, minibatch):
        pi = self.pi(minibatch)
        sigma = self.sigma(minibatch)
        # original sigma = torch.clamp(sigma, np.log(np.sqrt(1e-4)), 1e8)
        # working 
        sigma =  torch.clamp(sigma, np.log(np.sqrt(1e-3)), 5e1)
        #try 3 sigma = torch.clamp(sigma, np.log(np.sqrt(1e-3)), 1e4)
        sigma = sigma.view(-1, self.num_gaussians, self.out_features)
        mu = self.mu(minibatch)
        mu = mu.view(-1, self.num_gaussians, self.out_features)
        return pi, sigma, mu

    @staticmethod
    def gaussian_probability(sigma, x_mu, x):
        """Returns the probability of `data` given MoG parameters `sigma` and `mu`.
        Arguments:
            sigma (BxGxO): The standard deviation of the Gaussians. B is the batch
                size, G is the number of Gaussians, and O is the number of
                dimensions per Gaussian.
            mu (BxGxO): The means of the Gaussians. B is the batch size, G is the
                number of Gaussians, and O is the number of dimensions per Gaussian.
            data (BxI): A batch of data. B is the batch size and I is the number of
                input dimensions.
        Returns:
            probabilities (BxG): The probability of each point in the probability
                of the distribution in the corresponding sigma/mu index.
        """
        x = x.unsqueeze(1).expand_as(sigma)
        var = (torch.exp(sigma)**2)
        return -((x - x_mu)**2) / (2 * var + 1e-4) - sigma - math.log(
            math.sqrt(2 * math.pi))

    @staticmethod
    def mdn_loss(pi, sigma, mu, target):
        """Calculates the error, given the MoG parameters and the target
        The loss is the negative log likelihood of the data given the MoG
        parameters.
        """
        nll = log_sum_exp(pi[:, :, None] +
                          MDN.gaussian_probability(sigma, mu, target))
        nll = -torch.sum(nll, dim=-1)
        return torch.mean(nll)

    @staticmethod
    def sample(pi, sigma, mu):
        """Draw samples from a MoG.
        """
        categorical = Categorical(torch.exp(pi))
        pis = list(categorical.sample().data)
        sigma = torch.exp(sigma)
        sample = Variable(
            sigma.data.new(sigma.size(0), sigma.size(2)).normal_())
        for i, idx in enumerate(pis):
            sample[i] = sample[i].mul(sigma[i, idx]).add(mu[i, idx])
        return sample

In [ ]:
def initialize_weights(model):
    if type(model) in [nn.Linear]:
        nn.init.xavier_normal_(model.weight.data)
    elif type(model) in [nn.LSTM, nn.RNN, nn.GRU]:
        nn.init.xavier_normal_(model.weight_hh_l0)
        nn.init.xavier_normal_(model.weight_ih_l0)

In [ ]:
import torch.autograd as autograd

class SimpleRNN(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(SimpleRNN, self).__init__()
        # 32 was used for all the simulated data
        #hidden_dim = 32 #
        
        hidden_dim = 128 #hidden_dim

        #self.inp = torch.nn.Linear(n_features, hidden_size)
        num_layers = 2
        #self.rnn = LayerNormLSTM(n_features, hidden_dim, num_layers = num_layers)
        self.rnn = torch.nn.LSTM(n_features, hidden_dim, num_layers = num_layers)
        
        # 64 was used for all the simulated data
        #self.out = torch.nn.Linear(hidden_dim, 64)
        
        #self.out = torch.nn.Linear(hidden_dim, 32)
        self.mdn = MDN(hidden_dim, n_outputs, 5)

        
        #self.hidden = None
        
        initialize_weights(self.rnn)
        #initialize_weights(self.out)
        initialize_weights(self.mdn)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        if verbose:
            print("Step 0:")
            print(inputs.shape)
        inputs = inputs.permute([1, 0, 2])
        if verbose:
            print("Step 1:")
            print(inputs.shape)
        #self.rnn.flatten_parameters()
        output, hidden = self.rnn(inputs, hidden)
        output = output[-1, :, :] #output[:, :, :] #output[-1, :, :]
        #output = output.permute([1, 0, 2])
        if verbose:
            print("Step 3:")
            print(output.shape)
        output = output.squeeze()
        if verbose:
            print("Step 4:")
            print(output.shape)
        #output = self.out(output)
        if verbose:
            print("Step 5:")
            print(output.shape)
            print(output)
        output = self.mdn(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        if verbose:
            print("inputs size: ", inputs.size)
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
import torch.autograd as autograd
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from torch import nn
import seaborn as sns
import tables 
from time import sleep

from tqdm import tqdm_notebook
import tqdm


class HiddenClassifier(torch.nn.Module):

    def __init__(self, input_dim, output_dim):

        super(HiddenClassifier, self).__init__() 

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 256),
            torch.nn.ReLU(),
            #torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            #torch.nn.Dropout(0.2),
            torch.nn.Linear(128, output_dim),
            #torch.nn.Softmax()#torch.nn.Sigmoid()
        )
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, verbose = False):
        out = self.model(x)
        return out

In [ ]:
hmodel = HiddenClassifier(256, 1).cuda()
#hmodel = HiddenClassifier(256, 40).cuda()
#äweight = torch.tensor([0.00000001, 0.1 , 1 , 1]).cuda()
#criterion = nn.CrossEntropyLoss()# Mean Squared Loss
criterion = nn.MSELoss()# Mean Squared Loss
#criterion = nn.CrossEntropyLoss()
l_rate = 0.001 
#optimizer = torch.optim.SGD(hmodel.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(hmodel.parameters(), lr=0.001)
hmodel

In [ ]:
vision_bins = 16
#rnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
rnn_model = SimpleRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
#rnn = torch.load('asd.pt')
#rnn_model = torch.load('memsteprnn.pt')
rnn_model = torch.load('complexnew-rnn.pt')
rnn_model.eval()

In [ ]:
def make_one_hot(labels, C):
    A = np.zeros((labels.shape[0], C))
    for i in range(labels.shape[0]):
        A[i, min(C-1, labels[i].astype('int'))] = 1
    return A
    

In [ ]:
def group_wall_labels(labels):
    for i in range(labels.shape[0]):
        if labels[i] > 2:
            labels[i] = 3
    return labels

In [ ]:
runs_per_dataset

In [ ]:
from sklearn import metrics
import random
import time
import tables
import numpy as np
import time
%matplotlib inline
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
data_from_runs = []
#vision_bins = 16
#rnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
#rnn_model = SimpleRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
#rnn = torch.load('asd.pt')
#rnn_model = torch.load('memsteprnn.pt')
#rnn_model.eval()
eqltrain = np.asarray(eqltrain)
h5ftest = tables.open_file('pathtofile.h5', mode='r')
test = h5ftest.root.train[:, :, :]
print("lala test shape: ", test.shape)
data = []

for samples_per_class in intervalls:
    intervall_runs = []
    for runs_per_intervall in range(0,runs_per_dataset):
        print(runs_per_intervall + 1, ". run with ", samples_per_class, "samples per class")
        batch_size = 32
        train = []
        print("eqltrain shape", eqltrain.shape[0])
        train = eqltrain[np.random.choice(eqltrain.shape[0], samples_per_class*40), :, :]
        #train = eqltrain[:40000, :, :]
        hmodel = HiddenClassifier(256, 1).cuda()
        criterion = nn.MSELoss()# Mean Squared Loss
        l_rate = 0.001 
        optimizer = torch.optim.Adam(hmodel.parameters(), lr=0.001)
        train = np.asarray(train)
        np.random.shuffle(train) #inplace op
        print(train.shape)
        print("samples per classes: ", train.shape[0]/40)
        rec_losses = []
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
    
        epochs = int(max(np.round(20000 / train.shape[0]), 10))
        #TODO: MORE EPOCHS
        #epochs = int(max(np.round(20000 / train.shape[0]), 10))
        print("Epochs: ", epochs)
        tqdm_epochs = tqdm_notebook(list(range(epochs)))
        labels = []

        tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
                                   total= 1)#train.shape[0] // batch_size, leave = False)
        for epoch in range(0, epochs): #tqdm_epochs: 
            np.random.shuffle(train)
            #tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
            #                           total= train.shape[0] // batch_size, leave = False)#train.shape[0] // batch_size, leave = False)
            for minibatch_idx, (batch_data) in enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)):
            #for minibatch_idx, (batch_data) in tqdm_iter: 
                i = i+1
                batch_X = batch_data[:, :-1, 6+4:].astype(np.float32) 
                labels = batch_data[:, -1, 9].astype(np.float32) 
                #print(labels)
                #print(batch_data[:, -1, 6:12].astype(np.float32) )
                batch_Y = labels
                with autograd.detect_anomaly():
                    hmodel.zero_grad()
                    batch_X_rnn = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
                    batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
                    t, hidden_input = rnn_model.forward(batch_X_rnn, verbose = False)
                    #print(hidden_input[0].shape)
                    #print(hidden_input[1].shape)
                    batch_X = torch.cat((hidden_input[0][1], hidden_input[1][1]), 1)
                    batch_X = torch.autograd.Variable(batch_X).cuda()
                    outputs = hmodel.forward(batch_X) #847
                    loss = criterion(outputs.view(-1), batch_Y)
                    loss.backward()
                    optimizer.step()
                    y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
                    y_preds = np.concatenate((y_preds, y_pred))
                    y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
                    y_trues = np.concatenate((y_trues, y_true))
                    rec_losses.append(float(loss.cpu().data.numpy()))
                    tqdm_epochs.set_description("Loss {:6.5f}".format(np.mean(rec_losses[-100:])))   
        fig, ax = plt.subplots(1, 1, figsize=(16,4))
        ax.plot(pd.Series(rec_losses).rolling(100).mean())
        plt.show()
        time.sleep(2)
        clear_output()
        batch_size = 2
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
        label_counts = []

        for batch_data in iterate_data(test, batchsize=batch_size, shuffle=False):
            batch_X = batch_data[:, :-1, 6+4:].astype(np.float32)
            labels = batch_data[:, -1, 9].astype(np.float32)
            batch_Y = labels #group_wall_labels(labels)
            batch_X_rnn = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
            batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
            t, hidden_input = rnn_model.forward(batch_X_rnn, verbose = False)
            batch_X = torch.cat((hidden_input[0][1], hidden_input[1][1]), 1)
            batch_X = torch.autograd.Variable(batch_X).cuda()
            outputs = hmodel.forward(batch_X) #847
            loss = criterion(outputs.view(-1), batch_Y)
            y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
            y_preds = np.concatenate((y_preds, y_pred))
            y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
            y_trues = np.concatenate((y_trues, y_true))
        y_trues_oh = make_one_hot(np.round(y_trues), 40)
        y_preds_oh = make_one_hot(np.round(y_preds), 40)
    
        # Split the data into a training set and a test set
        y_test = np.argmax(y_trues_oh, axis = 1)
        y_pred = np.argmax(np.round(y_preds_oh), axis = 1)
        np.set_printoptions(precision=2)
        # Plot non-normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None)
        # Plot normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None, normalize=True)
        #plt.show()
    
        errs = np.abs(y_test-y_pred)
        plt.hist(errs)
        plt.title('Samples per class: {}'.format(samples_per_class))
        plt.show()
        time.sleep(2)
        print("error not bigger than 5: ", np.sum(errs < 5)/errs.shape[0])
    
        mean_err = np.sum(errs) / y_test.shape[0]
        print("Durchschnittliche Abweichung: ", mean_err)
    
        standard_derivation_err = np.sqrt(1/(y_test.shape[0]-1) * np.sum(np.square(mean_err-errs)))
        print("standard abweichung: ", standard_derivation_err)

        cm = metrics.confusion_matrix(np.argmax(y_trues_oh, axis = 1), np.argmax(np.round(y_preds_oh), axis = 1), labels=None, sample_weight=None)
        #print(cm.shape)    
        print("richtig klassifiziert: ", cm.trace()/y_test.shape[0])
        #data.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    
        plt.imshow(cm)
        plt.title('Samples per class: {}'.format(samples_per_class))
        plt.show()
        
        intervall_runs.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    data.append(np.asarray(intervall_runs))
    print("intervall_runs shape: ", len(intervall_runs))

In [ ]:
datameans = np.mean(np.asarray(data), axis = 1)

In [ ]:
datameans

In [ ]:
std_errorsr = []
std_correctsr = []
for intervall_data in data:
    print(intervall_data)
    leninterveall = intervall_data.shape[0]
    mean_error = np.mean(intervall_data[:, 1])
    mean_correct = np.mean(intervall_data[:, 3])
    std_errorsr.append(np.sqrt(1/leninterveall * 
                            np.sum((intervall_data[:, 1] - mean_error)
                                   *(intervall_data[:, 1] - mean_error))))
    std_correctsr.append(np.sqrt(1/leninterveall * 
                        np.sum((intervall_data[:, 3] - mean_correct)
                                *(intervall_data[:, 3] - mean_correct))))

In [ ]:
datanp = datameans
datanp

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datanp[:, 0], datanp[:, 1], 'ro')
plt.ylabel('some numbers')
plt.xscale('linear')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = datanp[:, 0]
y = datanp[:, 1]
e = datanp[:, 2]

plt.errorbar(x, y, e, linestyle='None', marker='^')

plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datanp[:, 0], datanp[:, 3]*100, 'ro-')
plt.ylabel('some numbers')
plt.xscale('linear')
plt.show()

# From scratch

In [ ]:
import torch.autograd as autograd

class SimpleClassifier(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(SimpleClassifier, self).__init__()
        
        hidden_dim = 128 #hidden_dim

        num_layers = 1
        self.rnn = torch.nn.LSTM(n_features, hidden_dim, num_layers = num_layers)
        #self.out = torch.nn.Linear(hidden_dim, n_outputs)
        #self.softmax = nn.Sigmoid()
        self.out = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, n_outputs),
            #torch.nn.Softmax()
        )
                    

        
        #self.hidden = None
        
        initialize_weights(self.rnn)
        initialize_weights(self.out)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        inputs = inputs.permute([1, 0, 2])
        output, hidden = self.rnn(inputs, hidden)
        output = output[-1, :, :] #output[:, :, :] #output[-1, :, :]
        output = output.squeeze()
        output = self.out(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        if verbose:
            print("inputs", inputs.shape)
            print("inputs size: ", inputs.size)
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
#constrnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
sclass_model = SimpleClassifier(133, 1).cuda()
#constrnn = torch.load('asdf.pt')
#constrnn = torch.load('constcomplex1rnntrain.pt')

#weight = torch.tensor([0.88675, 0.2187 , 3 , 3]).cuda()

#criterion = nn.CrossEntropyLoss(weight = weight)
#optimizer = torch.optim.SGD(sclass_model.parameters(), lr=0.001, momentum=0.9)

criterion = nn.MSELoss()# Mean Squared Loss
#criterion = nn.CrossEntropyLoss()
l_rate = 0.001 
#optimizer = torch.optim.SGD(hmodel.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(sclass_model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(sclass_model.parameters(), lr=0.005)
#optimizer = torch.optim.RMSprop(sclass_model.parameters())

In [ ]:
intervalls

In [ ]:
from sklearn import metrics
import random
import time
import tables
import numpy as np
import time
%matplotlib inline
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
data_from_runs = []


eqltrain = np.asarray(eqltrain)
h5ftest = tables.open_file('pathtofile.h5', mode='r')
test = h5ftest.root.train[:, :, :]
#print("lala test shape: ", test.shape)
datarawrnn = []

for samples_per_class in intervalls:
    intervall_runs = []
    for runs_per_intervall in range(0,runs_per_dataset):
        print(runs_per_intervall + 1, ". run with ", samples_per_class, "samples per class")
        batch_size = 32
        train = []
        train = eqltrain[np.random.choice(eqltrain.shape[0], samples_per_class*40), :, :]
        sclass_model = SimpleClassifier(133, 1).cuda()
        criterion = nn.MSELoss()# Mean Squared Loss
        l_rate = 0.001 
        optimizer = torch.optim.Adam(sclass_model.parameters(), lr=0.001)
        train = np.array(train)
        np.random.shuffle(train) #inplace op
        print(train.shape)
        print("samples per classes: ", train.shape[0]/40)
        rec_losses = []
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
    
        epochs = int(max(np.round(50000 / train.shape[0]), 10))
        #TODO: MORE EPOCHS
        #epochs = int(max(np.round(20000 / train.shape[0]), 10))
        print("Epochs: ", epochs)
        tqdm_epochs = tqdm_notebook(list(range(epochs)))
        labels = []

        tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
                                   total= 1)#train.shape[0] // batch_size, leave = False)
        for epoch in range(0, epochs): #tqdm_epochs: 
            np.random.shuffle(train)
            #tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
            #                           total= train.shape[0] // batch_size, leave = False)#train.shape[0] // batch_size, leave = False)
            for minibatch_idx, (batch_data) in enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)):
            #for minibatch_idx, (batch_data) in tqdm_iter: 
                i = i+1
                batch_X = batch_data[:, :-1, 6+4:].astype(np.float32) 
                #print("bx shape", batch_X.shape)
                labels = batch_data[:, -1, 9].astype(np.float32) 
                #print(labels)
                #print(batch_data[:, -1, 6:12].astype(np.float32) )
                batch_Y = labels
                #print("by shape", batch_Y.shape)
                with autograd.detect_anomaly():
                    sclass_model.zero_grad()
                    batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
                    batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
                    outputs, hidden_input = sclass_model.forward(batch_X, verbose = False)
                    loss = criterion(outputs.view(-1), batch_Y)
                    loss.backward()
                    optimizer.step()
                    y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
                    y_preds = np.concatenate((y_preds, y_pred))
                    y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
                    y_trues = np.concatenate((y_trues, y_true))
                    rec_losses.append(float(loss.cpu().data.numpy()))
                    tqdm_epochs.set_description("Loss {:6.5f}".format(np.mean(rec_losses[-100:])))   
        fig, ax = plt.subplots(1, 1, figsize=(16,4))
        ax.plot(pd.Series(rec_losses).rolling(100).mean())
        plt.show()
        time.sleep(2)
        clear_output()
        batch_size = 2
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
        label_counts = []

        for batch_data in iterate_data(test, batchsize=batch_size, shuffle=False):
            batch_X = batch_data[:, :-1, 6+4:].astype(np.float32)
            labels = batch_data[:, -1, 9].astype(np.float32)
            batch_Y = labels #group_wall_labels(labels)
            batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
            batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
            outputs, hidden_input = sclass_model.forward(batch_X, verbose = False)
            loss = criterion(outputs.view(-1), batch_Y)
            y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
            y_preds = np.concatenate((y_preds, y_pred))
            y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
            y_trues = np.concatenate((y_trues, y_true))
        y_trues_oh = make_one_hot(np.round(y_trues), 40)
        y_preds_oh = make_one_hot(np.round(y_preds), 40)
    
        # Split the data into a training set and a test set
        y_test = np.argmax(y_trues_oh, axis = 1)
        y_pred = np.argmax(np.round(y_preds_oh), axis = 1)
        np.set_printoptions(precision=2)
        # Plot non-normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None)
        # Plot normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None, normalize=True)
        #plt.show()
    
        errs = np.abs(y_test-y_pred)
        plt.hist(errs)
        plt.title('Samples per class: {}'.format(samples_per_class))
        plt.show()
        time.sleep(2)
        print("error not bigger than 5: ", np.sum(errs < 5)/errs.shape[0])
    
        mean_err = np.sum(errs) / y_test.shape[0]
        print("Durchschnittliche Abweichung: ", mean_err)
    
        standard_derivation_err = np.sqrt(1/(y_test.shape[0]-1) * np.sum(np.square(mean_err-errs)))
        print("standard abweichung: ", standard_derivation_err)

        cm = metrics.confusion_matrix(np.argmax(y_trues_oh, axis = 1), np.argmax(np.round(y_preds_oh), axis = 1), labels=None, sample_weight=None)
        #print(cm.shape)    
        print("richtig klassifiziert: ", cm.trace()/y_test.shape[0])
        #data.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    
        #plt.imshow(cm)
        #plt.title('Samples per class: {}'.format(samples_per_class))
        #plt.show()
        
        intervall_runs.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    datarawrnn.append(np.asarray(intervall_runs))
    print("intervall_runs shape: ", len(intervall_runs))

In [ ]:
std_errorrr = []
std_correctrr = []
for intervall_data in datarawrnn:
    print(intervall_data)
    leninterveall = intervall_data.shape[0]
    mean_error = np.mean(intervall_data[:, 1])
    mean_correct = np.mean(intervall_data[:, 3])
    std_errorrr.append(np.sqrt(1/leninterveall * 
                            np.sum((intervall_data[:, 1] - mean_error)
                                   *(intervall_data[:, 1] - mean_error))))
    std_correctrr.append(np.sqrt(1/leninterveall * 
                        np.sum((intervall_data[:, 3] - mean_correct)
                                *(intervall_data[:, 3] - mean_correct))))

In [ ]:
datanprawrnn = np.mean(np.asarray(datarawrnn), axis = 1)
datanprawrnn

In [ ]:
print(3)

In [ ]:
plt.figure(figsize=(20,10))
hiddenrnnline = plt.plot(datanp[:, 0], datanp[:, 1], 'ro', label="Hidden + RNN)")
rawrnnline = plt.plot(datanprawrnn[:, 0], datanprawrnn[:, 1], 'bo', label="Raw Data + RNN)")
plt.ylabel('Average error')
plt.xscale('linear')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20,10))
hiddenrnnline = plt.errorbar(datanp[:, 0], datanp[:, 1], datanp[:, 2], linestyle='None', marker='^')
rawrnnline = plt.errorbar(datanprawrnn[:, 0], datanprawrnn[:, 1], datanprawrnn[:, 2], linestyle='None', marker='^')

plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))

plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datanprawrnn[:, 0], datanprawrnn[:, 3]*100, 'ro-')
plt.ylabel('some numbers')
plt.xscale('linear')
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datanp[:, 0], datanp[:, 3]*100, 'ro-')
plt.plot(datanprawrnn[:, 0], datanprawrnn[:, 3]*100, 'bo-')
plt.ylabel('Richtig klassifiziert')
plt.xscale('linear')
plt.show()

# Simple

In [ ]:
import torch.autograd as autograd
import torch.nn.functional as F
import torch

class SC(torch.nn.Module):

    def __init__(self, input_dim, output_dim):

        super(SC, self).__init__() 
        
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, output_dim),
        )


    def forward(self, x, verbose = False):
        # Here the forward pass is simply a linear functio
        out = self.model(x)
        return out

In [ ]:
#constrnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
sc_model = SC(33*133, 1).cuda()
#constrnn = torch.load('asdf.pt')
#constrnn = torch.load('constcomplex1rnntrain.pt')

#weight = torch.tensor([0.88675, 0.2187 , 3 , 3]).cuda()
criterion = torch.nn.MSELoss()
#criterion = nn.MSELoss()
optimizer = torch.optim.Adam(sc_model.parameters(), lr=0.001)

In [ ]:
from sklearn import metrics
import random
import time
import tables
import numpy as np
import time
%matplotlib inline
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
data_from_runs = []


eqltrain = np.asarray(eqltrain)
h5ftest = tables.open_file('pathtofile.h5', mode='r')
test = h5ftest.root.train[:, :, :]
#print("lala test shape: ", test.shape)
datarawsimple = []

for samples_per_class in intervalls:
    intervall_runs = []
    for runs_per_intervall in range(0,runs_per_dataset):
        print(runs_per_intervall + 1, ". run with ", samples_per_class, "samples per class")
        batch_size = 32
        train = []
        train = eqltrain[np.random.choice(eqltrain.shape[0], samples_per_class*40), :, :]
        sc_model = SC(38*133, 1).cuda()
        criterion = nn.MSELoss()# Mean Squared Loss
        l_rate = 0.001 
        optimizer = torch.optim.Adam(sc_model.parameters(), lr=0.001)
        train = np.array(train)
        np.random.shuffle(train) #inplace op
        print(train.shape)
        print("samples per classes: ", train.shape[0]/40)
        rec_losses = []
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
    
        epochs = int(max(np.round(50000 / train.shape[0]), 10))
        #TODO: MORE EPOCHS
        #epochs = int(max(np.round(20000 / train.shape[0]), 10))
        print("Epochs: ", epochs)
        tqdm_epochs = tqdm_notebook(list(range(epochs)))
        labels = []

        tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
                                   total= 1)#train.shape[0] // batch_size, leave = False)
        for epoch in range(0, epochs): #tqdm_epochs: 
            np.random.shuffle(train)
            #tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
            #                           total= train.shape[0] // batch_size, leave = False)#train.shape[0] // batch_size, leave = False)
            for minibatch_idx, (batch_data) in enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)):
            #for minibatch_idx, (batch_data) in tqdm_iter: 
                i = i+1
                batch_X = batch_data[:, :-1, 6+4:].astype(np.float32) 
                batch_X = batch_X.reshape(batch_size,38*133)
                #print("bx shape", batch_X.shape)
                labels = batch_data[:, -1, 9].astype(np.float32) 
                #print(labels)
                #print(batch_data[:, -1, 6:12].astype(np.float32) )
                batch_Y = labels
                #print("by shape", batch_Y.shape)
                with autograd.detect_anomaly():
                    sc_model.zero_grad()
                    batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
                    batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
                    outputs = sc_model.forward(batch_X, verbose = False)
                    loss = criterion(outputs.view(-1), batch_Y)
                    loss.backward()
                    optimizer.step()
                    y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
                    y_preds = np.concatenate((y_preds, y_pred))
                    y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
                    y_trues = np.concatenate((y_trues, y_true))
                    rec_losses.append(float(loss.cpu().data.numpy()))
                    tqdm_epochs.set_description("Loss {:6.5f}".format(np.mean(rec_losses[-100:])))   
        fig, ax = plt.subplots(1, 1, figsize=(16,4))
        ax.plot(pd.Series(rec_losses).rolling(100).mean())
        plt.show()
        time.sleep(2)
        clear_output()
        batch_size = 2
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
        label_counts = []

        for batch_data in iterate_data(test, batchsize=batch_size, shuffle=False):
            batch_X = batch_data[:, :-1, 6+4:].astype(np.float32)
            batch_X = batch_X.reshape(batch_size,38*133)
            labels = batch_data[:, -1, 9].astype(np.float32)
            batch_Y = labels #group_wall_labels(labels)
            batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
            batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
            outputs = sc_model.forward(batch_X) #847
            loss = criterion(outputs.view(-1), batch_Y)
            y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
            y_preds = np.concatenate((y_preds, y_pred))
            y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
            y_trues = np.concatenate((y_trues, y_true))
        y_trues_oh = make_one_hot(np.round(y_trues), 40)
        y_preds_oh = make_one_hot(np.round(y_preds), 40)
    
        # Split the data into a training set and a test set
        y_test = np.argmax(y_trues_oh, axis = 1)
        y_pred = np.argmax(np.round(y_preds_oh), axis = 1)
        np.set_printoptions(precision=2)
        # Plot non-normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None)
        # Plot normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None, normalize=True)
        #plt.show()
    
        errs = np.abs(y_test-y_pred)
        plt.hist(errs)
        plt.title('Samples per class: {}'.format(samples_per_class))
        plt.show()
        time.sleep(2)
        print("error not bigger than 5: ", np.sum(errs < 5)/errs.shape[0])
    
        mean_err = np.sum(errs) / y_test.shape[0]
        print("Durchschnittliche Abweichung: ", mean_err)
    
        standard_derivation_err = np.sqrt(1/(y_test.shape[0]-1) * np.sum(np.square(mean_err-errs)))
        print("standard abweichung: ", standard_derivation_err)

        cm = metrics.confusion_matrix(np.argmax(y_trues_oh, axis = 1), np.argmax(np.round(y_preds_oh), axis = 1), labels=None, sample_weight=None)
        #print(cm.shape)    
        print("richtig klassifiziert: ", cm.trace()/y_test.shape[0])
        #data.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    
        #plt.imshow(cm)
        #plt.title('Samples per class: {}'.format(samples_per_class))
        #plt.show()
        
        intervall_runs.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
        #clear_output()
    datarawsimple.append(np.asarray(intervall_runs))
    print("intervall_runs shape: ", len(intervall_runs))

In [ ]:
std_errorrs = []
std_correctrs = []
for intervall_data in datarawsimple:
    print(intervall_data)
    leninterveall = intervall_data.shape[0]
    mean_error = np.mean(intervall_data[:, 1])
    mean_correct = np.mean(intervall_data[:, 3])
    std_errorrs.append(np.sqrt(1/leninterveall * 
                            np.sum((intervall_data[:, 1] - mean_error)
                                   *(intervall_data[:, 1] - mean_error))))
    std_correctrs.append(np.sqrt(1/leninterveall * 
                        np.sum((intervall_data[:, 3] - mean_correct)
                                *(intervall_data[:, 3] - mean_correct))))

In [ ]:
datanprawsimple = np.mean(np.asarray(datarawsimple), axis = 1)
datanprawrnn

In [ ]:
plt.figure(figsize=(20,10))
hiddenrnnline = plt.plot(datanp[:, 0], datanp[:, 1], 'ro-', label="Hidden + RNN)")
rawrnnline = plt.plot(datanprawrnn[:, 0], datanprawrnn[:, 1], 'bo-', label="Raw Data + RNN)")
rawsimpleline = plt.plot(datanprawsimple[:, 0], datanprawsimple[:, 1], 'go-', label="Raw Data + Simple)")
plt.ylabel('Average error')
plt.xscale('linear')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20,10))
hiddenrnnline = plt.errorbar(datanp[:, 0], datanp[:, 1], datanp[:, 2], linestyle='None', marker='^')
rawrnnline = plt.errorbar(datanprawrnn[:, 0], datanprawrnn[:, 1], datanprawrnn[:, 2], linestyle='None', marker='^')
rawsimpleline = plt.errorbar(datanprawsimple[:, 0], datanprawsimple[:, 1], datanprawsimple[:, 2], linestyle='None', marker='^')

plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))

plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datanp[:, 0], datanp[:, 3]*100, 'ro-')
plt.plot(datanprawrnn[:, 0], datanprawrnn[:, 3]*100, 'bo-')
plt.plot(datanprawsimple[:, 0], datanprawsimple[:, 3]*100, 'go-')
plt.ylabel('Richtig klassifiziert')
plt.xscale('linear')
plt.show()

# Simple last timestep

In [ ]:
import torch.autograd as autograd

class SCCC(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(SCCC, self).__init__()
        self.out = torch.nn.Sequential(
            torch.nn.Linear(n_features, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, n_outputs),
            #torch.nn.Softmax()
        )
            
        initialize_weights(self.out)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        output = inputs #inputs.flatten(start_dim = 1, end_dim = 2)
        output = self.out(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
#constrnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
sccc_model = SCCC(133, 1).cuda()
#constrnn = torch.load('asdf.pt')
#constrnn = torch.load('constcomplex1rnntrain.pt')

#weight = torch.tensor([0.88675, 0.2187 , 3 , 3]).cuda()

#criterion = nn.CrossEntropyLoss(weight = weight)
#optimizer = torch.optim.SGD(sclass_model.parameters(), lr=0.001, momentum=0.9)

criterion = nn.MSELoss()# Mean Squared Loss
#criterion = nn.CrossEntropyLoss()
l_rate = 0.001 
#optimizer = torch.optim.SGD(hmodel.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(sccc_model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(sclass_model.parameters(), lr=0.005)
#optimizer = torch.optim.RMSprop(sclass_model.parameters())

In [ ]:
from sklearn import metrics
import random
import time
import tables
import numpy as np
import time
%matplotlib inline
import time
import pylab as pl
from IPython import display
from IPython.display import clear_output
data_from_runs = []


eqltrain = np.asarray(eqltrain)
h5ftest = tables.open_file('pathtofile.h5', mode='r')
test = h5ftest.root.train[:, :, :]
#print("lala test shape: ", test.shape)
datarawsimplelast = []

for samples_per_class in intervalls:
    intervall_runs = []
    for runs_per_intervall in range(0,runs_per_dataset):
        print(runs_per_intervall + 1, ". run with ", samples_per_class, "samples per class")
        batch_size = 32
        train = []
        train = eqltrain[np.random.choice(eqltrain.shape[0], samples_per_class*40), :, :]
        sccc_model = SCCC(133, 1).cuda()
        criterion = nn.MSELoss()# Mean Squared Loss
        l_rate = 0.001 
        optimizer = torch.optim.Adam(sccc_model.parameters(), lr=0.001)
        train = np.array(train)
        np.random.shuffle(train) #inplace op
        print(train.shape)
        print("samples per classes: ", train.shape[0]/40)
        rec_losses = []
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
    
        epochs = int(max(np.round(50000 / train.shape[0]), 10))
        #TODO: MORE EPOCHS
        #epochs = int(max(np.round(20000 / train.shape[0]), 10))
        print("Epochs: ", epochs)
        tqdm_epochs = tqdm_notebook(list(range(epochs)))
        labels = []

        tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
                                   total= 1)#train.shape[0] // batch_size, leave = False)
        for epoch in range(0, epochs): #tqdm_epochs: 
            np.random.shuffle(train)
            #tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
            #                           total= train.shape[0] // batch_size, leave = False)#train.shape[0] // batch_size, leave = False)
            for minibatch_idx, (batch_data) in enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)):
            #for minibatch_idx, (batch_data) in tqdm_iter: 
                i = i+1
                batch_X = batch_data[:, -2, 6+4:].astype(np.float32) 
                #print("bx shape", batch_X.shape)
                labels = batch_data[:, -1, 9].astype(np.float32) 
                #print(labels)
                #print(batch_data[:, -1, 6:12].astype(np.float32) )
                batch_Y = labels
                #print("by shape", batch_Y.shape)
                with autograd.detect_anomaly():
                    sccc_model.zero_grad()
                    batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
                    batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
                    outputs, hidden_input = sccc_model.forward(batch_X, verbose = False)
                    loss = criterion(outputs.view(-1), batch_Y)
                    loss.backward()
                    optimizer.step()
                    y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
                    y_preds = np.concatenate((y_preds, y_pred))
                    y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
                    y_trues = np.concatenate((y_trues, y_true))
                    rec_losses.append(float(loss.cpu().data.numpy()))
                    tqdm_epochs.set_description("Loss {:6.5f}".format(np.mean(rec_losses[-100:])))   
        fig, ax = plt.subplots(1, 1, figsize=(16,4))
        ax.plot(pd.Series(rec_losses).rolling(100).mean())
        plt.show()
        time.sleep(2)
        clear_output()
        batch_size = 2
        y_preds = torch.empty((0, 1)).numpy()
        y_trues = torch.empty((0, 1)).numpy()
        label_counts = []

        #clear_output()
        for batch_data in iterate_data(test, batchsize=batch_size, shuffle=False):
            batch_X = batch_data[:, -2, 6+4:].astype(np.float32)
            labels = batch_data[:, -1, 9].astype(np.float32)
            batch_Y = labels #group_wall_labels(labels)
            batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
            batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
            outputs, hidden_input = sccc_model.forward(batch_X, verbose = False)
            loss = criterion(outputs.view(-1), batch_Y)
            y_pred = outputs.detach().cpu().numpy() #torch.argmax(outputs[:, :], 1).cpu().numpy()
            y_preds = np.concatenate((y_preds, y_pred))
            y_true = batch_Y.cpu().numpy()[:, None] #make_one_hot(batch_data[:, -1, 9].astype(np.float32), 40) 
            y_trues = np.concatenate((y_trues, y_true))
        y_trues_oh = make_one_hot(np.round(y_trues), 40)
        y_preds_oh = make_one_hot(np.round(y_preds), 40)
    
        # Split the data into a training set and a test set
        y_test = np.argmax(y_trues_oh, axis = 1)
        y_pred = np.argmax(np.round(y_preds_oh), axis = 1)
        np.set_printoptions(precision=2)
        # Plot non-normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None)
        # Plot normalized confusion matrix
        #plot_confusion_matrix(y_test, y_pred, classes=None, normalize=True)
        #plt.show()
    
        errs = np.abs(y_test-y_pred)
        plt.hist(errs)
        plt.title('Samples per class: {}'.format(samples_per_class))
        plt.show()
        time.sleep(2)
        print("error not bigger than 5: ", np.sum(errs < 5)/errs.shape[0])
    
        mean_err = np.sum(errs) / y_test.shape[0]
        print("Durchschnittliche Abweichung: ", mean_err)
    
        standard_derivation_err = np.sqrt(1/(y_test.shape[0]-1) * np.sum(np.square(mean_err-errs)))
        print("standard abweichung: ", standard_derivation_err)

        cm = metrics.confusion_matrix(np.argmax(y_trues_oh, axis = 1), np.argmax(np.round(y_preds_oh), axis = 1), labels=None, sample_weight=None)
        #print(cm.shape)    
        print("richtig klassifiziert: ", cm.trace()/y_test.shape[0])
        #data.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    
        #plt.imshow(cm)
        #plt.title('Samples per class: {}'.format(samples_per_class))
        #plt.show()
        
        intervall_runs.append(np.array([samples_per_class, mean_err, standard_derivation_err, cm.trace()/y_test.shape[0]]))
    datarawsimplelast.append(np.asarray(intervall_runs))
    print("intervall_runs shape: ", len(intervall_runs))

In [ ]:
datanprawsimplelast = np.mean(np.asarray(datarawsimplelast), axis = 1)
datanprawsimplelast

In [ ]:
std_errorls = []
std_correctls = []
for intervall_data in datarawsimplelast:
    print(intervall_data)
    leninterveall = intervall_data.shape[0]
    mean_error = np.mean(intervall_data[:, 1])
    mean_correct = np.mean(intervall_data[:, 3])
    std_errorls.append(np.sqrt(1/leninterveall * 
                            np.sum((intervall_data[:, 1] - mean_error)
                                   *(intervall_data[:, 1] - mean_error))))
    std_correctls.append(np.sqrt(1/leninterveall * 
                        np.sum((intervall_data[:, 3] - mean_correct)
                                *(intervall_data[:, 3] - mean_correct))))

In [ ]:
rawsimplelastline = plt.plot(datanprawsimplelast[:, 0], datanprawsimplelast[:, 1], 'co-', label="Last Step + Simple)")


In [ ]:
plt.figure(figsize=(20,10))
hiddenrnnline = plt.plot(datanp[:, 0] * 40, datanp[:, 1], 'ro-', label="Hidden + Simple")
rawrnnline = plt.plot(datanprawrnn[:, 0] * 40, datanprawrnn[:, 1], 'bo-', label="Raw Data + RNN")
rawsimpleline = plt.plot(datanprawsimple[:, 0] * 40, datanprawsimple[:, 1], 'go-', label="Raw Data + Simple")
rawsimplelastline = plt.plot(datanprawsimplelast[:, 0] *40, datanprawsimplelast[:, 1], 'co-', label="Last Step + Simple")

plt.ylabel('Average error')
plt.xlabel('Number of samples used for training')
plt.xscale('log')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))
plt.grid(True,which="both",ls="-")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
hiddenrnnline = plt.plot(datanp[:, 0] * 40, datanp[:, 1], 'ro-', label="Hidden + Simple")
rawrnnline = plt.plot(datanprawrnn[:, 0] * 40, datanprawrnn[:, 1], 'bo-', label="Raw Data + RNN")
rawsimpleline = plt.plot(datanprawsimple[:, 0] * 40, datanprawsimple[:, 1], 'go-', label="Raw Data + Simple")
rawsimplelastline = plt.plot(datanprawsimplelast[:, 0] *40, datanprawsimplelast[:, 1], 'co-', label="Last Step + Simple")

plt.ylabel('Average error')
plt.xlabel('Number of samples used for training')
plt.xscale('log')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))
plt.grid(True,which="both",ls="-")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
hiddenrnnline = plt.plot(datanp[:, 0] * 40, datanp[:, 2], 'ro-', label="Hidden + Simple")
rawrnnline = plt.plot(datanprawrnn[:, 0] * 40, datanprawrnn[:, 2], 'bo-', label="Raw Data + RNN")
rawsimpleline = plt.plot(datanprawsimple[:, 0] * 40, datanprawsimple[:, 2], 'go-', label="Raw Data + Simple")
rawsimplelastline = plt.plot(datanprawsimplelast[:, 0] * 40, datanprawsimplelast[:, 2], 'co-', label="Last Step + Simple")

plt.ylabel('Average error')
plt.xlabel('Number of samples used for training')
plt.xscale('log')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))
plt.grid(True,which="both",ls="-")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
hiddenrnnline = plt.plot(datanp[:, 0] * 40, datanp[:, 2], 'ro-', label="Hidden + Simple")
rawrnnline = plt.plot(datanprawrnn[:, 0] * 40, datanprawrnn[:, 2], 'bo-', label="Raw Data + RNN")
rawsimpleline = plt.plot(datanprawsimple[:, 0] * 40, datanprawsimple[:, 2], 'go-', label="Raw Data + Simple")
rawsimplelastline = plt.plot(datanprawsimplelast[:, 0] * 40, datanprawsimplelast[:, 2], 'co-', label="Last Step + Simple")

plt.ylabel('Average error')
plt.xlabel('Number of samples used for training')
plt.xscale('log')

plt.legend() #(loc=2, fontsize="small")
#plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))
plt.grid(True,which="both",ls="-")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20,10))
hiddenrnnline = plt.errorbar(datanp[:, 0], datanp[:, 1], datanp[:, 2], linestyle='None', marker='^')
rawrnnline = plt.errorbar(datanprawrnn[:, 0], datanprawrnn[:, 1], datanprawrnn[:, 2], linestyle='None', marker='^')
rawsimpleline = plt.errorbar(datanprawsimple[:, 0], datanprawsimple[:, 1], datanprawsimple[:, 2], linestyle='None', marker='^')
rawsimplelastline = plt.errorbar(datanprawsimplelast[:, 0], datanprawsimplelast[:, 1], datanprawsimple[:, 2], linestyle='None', marker='^')
plt.xscale('log')
plt.legend((hiddenrnnline, rawrnnline), ('Hidden + RNN', 'Raw Data + RNN'))
plt.grid(True,which="both",ls="-")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(datanp[:, 0] * 40, datanp[:, 3]*100, 'ro-', label="Hidden + Simple")
plt.plot(datanprawrnn[:, 0] * 40, datanprawrnn[:, 3]*100, 'bo-', label="Raw Data + RNN")
plt.plot(datanprawsimple[:, 0] * 40, datanprawsimple[:, 3]*100, 'go-', label="Raw Data + Simple")
plt.plot(datanprawsimplelast[:, 0] * 40, datanprawsimplelast[:, 3]*100, 'co-', label="Last Step + Simple")
plt.xlabel('Number of samples used for training')
plt.ylabel('Correctly classified in %')
plt.xscale('log')

plt.grid(True,which="both",ls="-")
plt.legend()
plt.show()
